In [2]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import scipy
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from nuclei_segmentation import otsu as ot
from nuclei_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: <object object at 0x00000275FDB8EEE0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Helen Jade\anaconda3\envs\dataanalysis0104_Env\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [6]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

## NIH3T3 Dataset

In [12]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [14]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

In [7]:
print(gt_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

In [8]:
print(img_NIH3T3)

[array([[0.14509805, 0.15294118, 0.14901961, ..., 0.07843138, 0.07843138,
        0.07450981],
       [0.16078432, 0.14901961, 0.14901961, ..., 0.07843138, 0.08235294,
        0.08235294],
       [0.14901961, 0.14901961, 0.15686275, ..., 0.08235294, 0.08235294,
        0.07843138],
       ...,
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09019608],
       [0.08627451, 0.09019608, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09411765],
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09411765,
        0.09019608]], dtype=float32), array([[0.08235294, 0.08235294, 0.08235294, ..., 0.12941177, 0.1254902 ,
        0.13333334],
       [0.07843138, 0.08235294, 0.08627451, ..., 0.12941177, 0.13333334,
        0.1254902 ],
       [0.08235294, 0.08627451, 0.08235294, ..., 0.12941177, 0.12941177,
        0.13333334],
       ...,
       [0.13333334, 0.13333334, 0.12156863, ..., 0.08627451, 0.08235294,
        0.08235294],
       [0.14901961

#### Median filter

In [52]:
# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
filter = [3, 4, 5, 10, 15, 20]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[j]))
    print("filtersize done")    

filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


In [44]:
#mit 2 D array geht so aber-> otsu nicht mehr möglich


# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
#filter = [3, 4, 5]
#median3_img_NIH3T3 = []
#median4_img_NIH3T3 = []
#median5_img_NIH3T3 = []

#for j in range (0, len(filter)):
   # median_img_NIH3T3 = []
    #for i in range(0, len(copy_img_NIH3T3)):
    #    median3_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[0]))
    #    median4_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[1]))
    #    median5_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[2]))
    #median_img_NIH3T3.append(median3_img_NIH3T3) 
    #median_img_NIH3T3.append(median4_img_NIH3T3)
    #median_img_NIH3T3.append(median5_img_NIH3T3)

    #print("filtersize done")  

filtersize done
filtersize done
filtersize done


3


#### gaussian filter

In [ ]:
# Apply gaussian filter to all images of each dataset
#filter größen die mach möchte 
filter_g = [3, 4, 5, 10, 15, 20]
gauss_img_NIH3T3 = []

for j in range (0, len(filter_g)):

    for i in range(0,len(copy_img_NIH3T3)):
        gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , filter_g[j]))
    print("filtersize done")
        


## Otsu thresholding

In [53]:
# Apply otsu thresholding to all images of each dataset - median filter
otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [i] , 256))




In [48]:
#otsu median mit 2D array-> nicht möglich

#otsu_median_img_NIH3T3 = []
#otsu3_median_img_NIH3T3 = []
#otsu4_median_img_NIH3T3 = []
#otsu5_median_img_NIH3T3 = []

#for i in range(0,len(median_img_NIH3T3)):
    #otsu3_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [0] , 256))
    #otsu4_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [1] , 256))
    #otsu5_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [2] , 256))
#otsu_median_img_NIH3T3.append(otsu3_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu4_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu5_median_img_NIH3T3)



AttributeError: 'list' object has no attribute 'flatten'

In [ ]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


6


## Dice score

In [54]:
# Dice score - NIH3T3-dataset
## median filter
dice_otsu_median_img_NIH3T3 = []
for j in range (0, len(gt_NIH3T3)):

    for i in range(0,len(otsu_median_img_NIH3T3)):
     dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[i] , gt_NIH3T3[j]))
    print(dice_otsu_median_img_NIH3T3)


#dice_otsu_median_img_NIH3T3 = []
#for i in range (0, len (gt6_NIH3T3)):
   # for j in range( 0, len(otsu_median_img_NIH3T3)):
     #   dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[j] , gt6_NIH3T3[i]))
#print(dice_otsu_median_img_NIH3T3)



[0.9131440698604878, 0.1693720666222276, 0.19193484084407544, 0.19049441590949104, 0.18228644043113887, 0.12653026281996405, 0.1772398704350131, 0.1510187658286041, 0.03376164961019833, 0.15015783916271386, 0.006350570223402593, 0.19496338352250003, 0.07481387866790437, 0.23193681083358023, 0.21220032352029558, 0.11654234313295304, 0.010211983889076216, 0.1778399567270129, 0.8977948217808237, 0.17012935747764263, 0.18779114142998604, 0.1913013626784403, 0.18334367689883993, 0.12709921325993925, 0.1935034220054836, 0.1464624434027136, 0.03440876574083759, 0.1504724617976209, 0.006239920653824978, 0.19562039339124418, 0.07495489585041824, 0.23326408751066285, 0.23118612883227008, 0.11920973556643645, 0.010720671464247189, 0.17940493234693047, 0.8946808298799254, 0.1692885429468797, 0.19193812983360675, 0.19027954682576925, 0.18229476516927057, 0.10898224644049921, 0.2071752992989288, 0.14483971853010164, 0.03137930041376132, 0.14979473040894964, 0.006352704001500269, 0.1946937906966847, 

KeyboardInterrupt: 

In [5]:

gt1_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt1_NIH3T3.append(imread(os.path.join(path , f)))

print(len(gt1_NIH3T3))

18


In [10]:
gt2_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt2_NIH3T3.append(imread(os.path.join(path , f)))
    gt2_NIH3T3.append(imread(os.path.join(path , f)))


print(len(gt1_NIH3T3))

2


In [38]:
print(len(gt_NIH3T3)) #<- muss irgendwie auch 108 lang, bild muss sich quasi immer wiederhoen
print(len(gt6_NIH3T3))
print(len(median_img_NIH3T3))

18
6
108


In [48]:
print(otsu_median_img_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)

NameError: name 'otsu_gauss_img_N2DH_GOWT1' is not defined